In [176]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.preprocessing import OneHotEncoder
import re

In [177]:
import warnings
warnings.filterwarnings('ignore')

In [178]:
df = pd.read_csv('../../data/processed/data_nun_cleared(2_of_2).csv')

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19737 entries, 0 to 19736
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                19737 non-null  int64  
 1   ID  объявления            19737 non-null  int64  
 2   Ремонт                    19737 non-null  object 
 3   Площадь комнат, м2        19737 non-null  object 
 4   Балкон                    19737 non-null  object 
 5   Окна                      19737 non-null  object 
 6   Санузел                   19737 non-null  object 
 7   Можно с детьми/животными  19737 non-null  object 
 8   Дополнительно             19465 non-null  object 
 9   Высота потолков, м        19737 non-null  float64
 10  Лифт                      19737 non-null  object 
 11  Мусоропровод              19737 non-null  object 
 12  Ссылка на объявление      19737 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 2.0+ MB


In [180]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [181]:
df['Ремонт'].value_counts()

Ремонт
Косметический    8410
Евроремонт       8160
Дизайнерский     3031
Без ремонта       136
Name: count, dtype: int64

In [182]:
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(df[['Ремонт']])

In [183]:
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Ремонт']))

In [184]:
# выкидываю столбец "без ремонта", чтобы не плодить лишние данные
encoded_df.drop('Ремонт_Без ремонта', inplace=True, axis=1)

In [185]:
encoded_df.columns = ['rem_diz', 'rem_evro', 'rem_kosm']
df_final = pd.concat([df['ID  объявления'], encoded_df], axis=1)
df_final.rename(columns={'ID  объявления': 'id'}, inplace=True)

In [186]:
# смотрим, какие символы нас ждут в площади
set(''.join(df['Площадь комнат, м2'].unique().tolist()))

{' ',
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '²',
 'м',
 '—'}

In [187]:
def get_rooms_size(text: str):
    # меняем все запятые на точки для корректного отображения чисел
    # но если точки есть, то считаем запятые разделителями
    
    # работа с дебильными значениями типо 18,13,10    
    if '.' not in text and ' ()+\-/—' in text:
        text = text.replace(',', '.')
    # убираем лишнюю фигню
    for char in 'м²':
        text = text.replace(char, '')
    # все возможные разделители текста
    delimiters = r'[ ()+\-/—,]'
    result = re.split(delimiters, text)
    result = [item for item in result if item]
    return sum((map(float, result)))

In [ ]:
# кто-то указал номер телефона в площади... (берем инфу из площади)
df.loc[7136, 'Площадь комнат, м2'] = '20'

In [213]:
df_final['rooms_size'] = df['Площадь комнат, м2'].apply(get_rooms_size)

In [221]:
df['Балкон'].value_counts()

Балкон
Нет                       6630
Балкон (1)                6336
Лоджия (1)                5032
Балкон (1), Лоджия (1)     610
Лоджия (2)                 537
Балкон (2)                 423
Балкон (3)                  45
Лоджия (3)                  42
Балкон (2), Лоджия (2)      21
Балкон (1), Лоджия (2)      21
Балкон (2), Лоджия (1)      16
Лоджия (4)                   5
Балкон (4)                   4
Балкон (3), Лоджия (1)       4
Балкон (1), Лоджия (3)       4
Балкон (2), Лоджия (3)       3
Балкон (1), Лоджия (4)       2
Балкон (3), Лоджия (3)       1
Балкон (4), Лоджия (4)       1
Name: count, dtype: int64

In [222]:
def balcony_values(description):
    balcony_match = re.search(r'Балкон \((\d+)\)', description)
    loggia_match = re.search(r'Лоджия \((\d+)\)', description)
    balcony = int(balcony_match.group(1)) if balcony_match else 0
    loggia = int(loggia_match.group(1)) if loggia_match else 0
    return pd.Series([balcony, loggia])

In [227]:
df_final[['balcony', 'logia']] = df['Балкон'].apply(balcony_values)

In [230]:
df['Окна'].value_counts()

Окна
Во двор            14374
На улицу и двор     3246
На улицу            2117
Name: count, dtype: int64

In [233]:
def view_encode(view):
    if view == 'Во двор':
        return pd.Series([1, 0])
    elif view == 'На улицу':
        return pd.Series([0, 1])
    else:
        return pd.Series([1, 1])

In [234]:
df_final[['dvor_view', 'street_view']] = df['Окна'].apply(view_encode)

In [245]:
def toilet_values(description):
    merged_match = re.search(r'Совмещенный \((\d+)\)', description)
    split_match = re.search(r'Раздельный \((\d+)\)', description)
    merged = int(merged_match.group(1)) if merged_match else 0
    split = int(split_match.group(1)) if split_match else 0
    return pd.Series([merged, split])

In [246]:
df_final[['merged_toilet', 'split_toilet']] = df['Санузел'].apply(toilet_values)

In [250]:
df['Можно с детьми/животными'].value_counts()

Можно с детьми/животными
Можно с детьми                       8540
Можно с детьми, Можно с животными    6075
Ни с кем нельзя!                     4915
Можно с животными                     207
Name: count, dtype: int64

In [251]:
def allow_encode(allow):
    if allow == 'Можно с детьми':
        return pd.Series([1, 0])
    elif allow == 'Можно с животными':
        return pd.Series([0, 1])
    elif allow == 'Можно с детьми, Можно с животными':
        return pd.Series([1, 1])
    else:
        return pd.Series([0, 0])

In [253]:
df_final[['kids_allowed', 'pets_allowed']] = df['Можно с детьми/животными'].apply(allow_encode)

In [269]:
df['Дополнительно'].fillna('', inplace=True)

In [270]:
def encode_features(equipment_list):
    equipment_items = equipment_list.split(', ')
    features = [
        'Мебель в комнатах', 'Мебель на кухне', 'Ванна', 'Душевая кабина', 
        'Стиральная машина', 'Кондиционер', 'Посудомоечная машина', 
        'Телевизор', 'Холодильник', 'Интернет', 'Телефон'
    ]
    encoded_features = {feature: (1 if feature in equipment_items else 0) for feature in features}
    return pd.Series(encoded_features)

In [276]:
df_final[['furn_in_rooms', 'furn_at_kitchen', 'bath',
        'shower', 'washing', 'aircon', 'dishwaser',
        'tv', 'freezer', 'internet', 'phone']] = df['Дополнительно'].apply(encode_features)

In [287]:
# убираем выбросы
df.loc[df['Высота потолков, м'] > 150, 'Высота потолков, м'] = df['Высота потолков, м'] / 100
df.loc[df['Высота потолков, м'] > 15, 'Высота потолков, м'] = df['Высота потолков, м'] / 10

In [291]:
# убираем жилье хоббита
df.loc[df['Высота потолков, м'] == 1.2, 'Высота потолков, м'] = df['Высота потолков, м'].median()

In [294]:
# и великана
df.loc[df['Высота потолков, м'] == 12.8, 'Высота потолков, м'] = df['Высота потолков, м'].median()

In [298]:
df_final['ceiling_height'] = df['Высота потолков, м']

In [305]:
def lift_values(description):
    pass_match = re.search(r'Пасс \((\d+)\)', description)
    gruz_match = re.search(r'Груз \((\d+)\)', description)
    pass_lift = int(pass_match.group(1)) if pass_match else 0
    gruz_lift = int(gruz_match.group(1)) if gruz_match else 0
    return pd.Series([pass_lift, gruz_lift])

In [306]:
df_final[['pass_lift', 'gruz_lift']] = df['Лифт'].apply(lift_values)

In [309]:
df_final['trashprovod'] = df['Мусоропровод'].apply(lambda x: 1 if x == 'Да' else 0)

In [316]:
# разбираемся с выбросами
df_final.loc[df_final['pass_lift'] > 10, 'pass_lift'] = df_final['pass_lift'] / 10
df_final.loc[df_final['gruz_lift'] == 12, 'gruz_lift'] = 1

In [317]:
df_final.describe()

,id,rem_diz,rem_evro,rem_kosm,rooms_size,balcony,logia,dvor_view,street_view,merged_toilet,...,aircon,dishwaser,tv,freezer,internet,phone,ceiling_height,pass_lift,gruz_lift,trashprovod
count,"19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00",...,"19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00","19,737.00"
mean,"267,114,888.22",0.15,0.41,0.43,39.76,0.41,0.35,0.89,0.27,0.75,...,0.40,0.34,0.64,0.86,0.79,0.20,2.76,1.48,0.41,0.79
std,"19,801,055.08",0.36,0.49,0.49,33.76,0.55,0.55,0.31,0.44,0.70,...,0.49,0.47,0.48,0.35,0.41,0.40,0.24,0.73,0.64,0.41
min,"107,298,592.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00
25%,"271,221,229.00",0.00,0.00,0.00,20.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,1.00,0.00,2.64,1.00,0.00,1.00
50%,"273,928,403.00",0.00,0.00,0.00,31.00,0.00,0.00,1.00,0.00,1.00,...,0.00,0.00,1.00,1.00,1.00,0.00,2.64,1.00,0.00,1.00
75%,"274,697,333.00",0.00,1.00,1.00,44.80,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,0.00,2.80,2.00,1.00,1.00
max,"275,006,443.00",1.00,1.00,1.00,"1,410.00",4.00,4.00,1.00,1.00,4.00,...,1.00,1.00,1.00,1.00,1.00,1.00,6.00,8.00,8.00,1.00


In [318]:
# экспортируем
df_final.to_csv('../../data/processed/data_final(2_of_2).csv')